In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
from scipy import stats
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv

In [2]:
rlist = []
records = os.path.normpath('mit-bih-dataframes/subject_list.csv')
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not    
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [3]:
subject_dataframes = []
for x in tqdm(rlist):
    subject_dataframes.append(pd.read_csv(os.path.normpath('mit-bih-dataframes/'+x+'.csv'), index_col=0))

100%|███████████████████████████████████████████| 23/23 [01:05<00:00,  2.86s/it]


In [4]:
subject_rpeaks = []
for x in tqdm(rlist):
    subject_rpeaks.append(pd.read_csv(os.path.normpath('mit-bih-extracted/'+x+'_rpeaks.csv'), names=['rpeaks']))

print(subject_rpeaks[0].head())

100%|███████████████████████████████████████████| 23/23 [00:00<00:00, 80.41it/s]

   rpeaks
0    61.0
1   200.0
2   358.0
3   584.0
4   729.0


In [5]:
subject_rr_rhythms = []
for x in tqdm(rlist):
    df = pd.read_csv(os.path.normpath('mit-bih-extracted/'+x+'_rr_ints.csv'), names=['rr_int'])
    df["rr_int_seconds"] = df["rr_int"] / 250 # 250 samples per second
    subject_rr_rhythms.append(df)

100%|██████████████████████████████████████████| 23/23 [00:00<00:00, 281.27it/s]


In [6]:
rhythms = []
for x in tqdm(range(len(subject_rpeaks))):
    rhythms = []
    for row in subject_rpeaks[x]['rpeaks']:
        if subject_dataframes[x]['AFIB'][row]:
            rhythms.append('A')
        elif subject_dataframes[x]['Normal'][row]:
            rhythms.append('N')
        elif subject_dataframes[x]['Other'][row]:
            rhythms.append('O')
    subject_rr_rhythms[x]["rhythms"] = rhythms[1:]

subject_rr_rhythms[0].head()

100%|███████████████████████████████████████████| 23/23 [00:11<00:00,  2.02it/s]


,rr_int,rr_int_seconds,rhythms
0,139,0.556,N
1,158,0.632,N
2,226,0.904,N
3,145,0.580,N
4,144,0.576,N


In [7]:
for x in tqdm(range(len(subject_rr_rhythms))):
    subject_rr_rhythms[x].to_csv(os.path.normpath('mit-bih-extracted/'+rlist[x]+'_rr_rhythms.csv'))

100%|███████████████████████████████████████████| 23/23 [00:02<00:00, 11.33it/s]
